In [1]:
import tqdm.notebook

from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path
from preprocessing import peyepeline

In [2]:
#data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'
data_collection_name = 'MultiplEYE_SL_SI_Ljubljana_1_2025'


If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [3]:
prepare_language_folder(data_collection_name)

In [4]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)
multipleye_sq

Folder test_sessions does not match the regex pattern \d\d\d_SL_SI_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SL_SI_1_ET\d. Not considered as session.


Parsing participant data : 130_SL_SI_1_ET1: 100%|██████████| 12/12 [00:00<00:00, 542.32it/s]


data_collection_name	MultiplEYE_SL_SI_Ljubljana_1_2025
num_sessions	12
num_pilots	0

In [5]:
preprocessed_data_folder = this_repo / "preprocessed_data" / data_collection_name
preprocessed_data_folder.mkdir(parents=True, exist_ok=True)

In [6]:
sessions = [s for s in multipleye_sq]

for sess in tqdm.notebook.tqdm(sessions[:3]):
    asc = sess['asc_path']
    output_folder = preprocessed_data_folder / sess['session_identifier']
    output_folder.mkdir(parents=True, exist_ok=True)

    gaze = peyepeline.load_gaze_data(
        asc_file=asc,
        lab_config=sess['lab_config'],
        session_idf=sess['session_identifier'],
        output_dir=output_folder,
        save=True
	)

    peyepeline.detect_fixation_and_saccades(
        gaze,
		output_dir=output_folder,
		save=True,
        session_idf=sess['session_identifier'],
    )


  0%|          | 0/3 [00:00<?, ?it/s]

/Users/debor/repos/multipleye-preprocessing/.venv/lib/python3.11/site-packages/pymovements/gaze/gaze.py:1786: UserWarning: Gaze contains samples but no components could be inferred. 
This usually happens if you did not specify any column content and the content could not be autodetected from the column names. 
Please specify 'pixel_columns', 'position_columns', 'velocity_columns' or 'acceleration_columns' explicitly during initialization. Otherwise, transformation methods may fail.
  warnings.warn(


TypeError: 'NoneType' object is not subscriptable

Step 1 pf peyepline: create the gaze frame.

	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)